In [22]:
from pykrx import stock
import numpy as np
import pandas as pd

In [2]:
df = stock.get_index_ohlcv_by_date("20200101","20200831","1001")
df = df[ [ '종가' ] ]
df.head()

코스피,종가
날짜,
2020-01-02,2175.17
2020-01-03,2176.46
2020-01-06,2155.07
2020-01-07,2175.54
2020-01-08,2151.31


# 이평선 매매
이평선을 돌파하면 매수 / 그렇지 않으면 매도

In [3]:
df['ma'] = df['종가'].rolling(3).mean().shift(1)
df.head()

코스피,종가,ma
날짜,,
2020-01-02,2175.17,NaN
2020-01-03,2176.46,NaN
2020-01-06,2155.07,NaN
2020-01-07,2175.54,2168.900000
2020-01-08,2151.31,2169.023333


In [4]:
cond = df['종가'] > df['ma']
df['status'] = np.where(cond, 1, 0)

# status 1 : 종가가 이동평균보다 크기 때문에 당일 매수
# status 0 : 종가가 이동평균보다 작기 때문에 당일 매도

df.iloc[-1,-1] = 0
df.tail()

코스피,종가,ma,status
날짜,,,
2020-08-25,2366.73,2302.880000,1
2020-08-26,2369.32,2333.716667,1
2020-08-27,2344.45,2355.293333,0
2020-08-28,2353.80,2360.166667,0
2020-08-31,2326.17,2355.856667,0


In [5]:
buy_condition = (df['status'] == 1) & (df['status'].shift(1) != 1)
sell_condition = (df['status'] == 0) & (df['status'].shift(1) == 1)

In [6]:
df[buy_condition].head()

코스피,종가,ma,status
날짜,,,
2020-01-07,2175.54,2168.900000,1
2020-01-09,2186.45,2160.640000,1
2020-01-22,2267.25,2250.966667,1
2020-02-04,2157.90,2128.630000,1
2020-02-11,2223.12,2213.653333,1


In [7]:
df[sell_condition].head()

코스피,종가,ma,status
날짜,,,
2020-01-08,2151.31,2169.023333,0
2020-01-21,2239.69,2253.753333,0
2020-01-23,2246.13,2256.526667,0
2020-02-10,2201.07,2201.840000,0
2020-02-18,2208.88,2239.573333,0


In [8]:
ROR = df.loc[sell_condition, '종가'].reset_index(drop=True) / df.loc[buy_condition, '종가'].reset_index(drop=True)
ROR.cumprod().iloc[-1]

1.2623991181070566

In [13]:
def single_moving_avg_ror(df, N):
    df = df[ [ '종가' ] ].copy()
    df['ma'] = df['종가'].rolling(N).mean().shift(1)
    cond = df['종가'] > df['ma']
    df['status'] = np.where(cond, 1, 0)
    df.iloc[-1,-1] = 0
    buy_condition = (df['status'] == 1) & (df['status'].shift(1) != 1)
    sell_condition = (df['status'] == 0) & (df['status'].shift(1) == 1)
    ROR = df.loc[sell_condition, '종가'].reset_index(drop=True) / df.loc[buy_condition, '종가'].reset_index(drop=True)
    ROR = ROR - 0.002
    return ROR.cumprod().iloc[-1]

In [14]:
single_moving_avg_ror(df,3)

1.2037220606995058

In [15]:
index = range(2,61)
result = [single_moving_avg_ror(df,x) for x in index]
result

[1.1493870023456916,
 1.2037220606995058,
 1.2038328802423408,
 1.2046567097489385,
 1.1839309166834184,
 1.2649700027492599,
 1.1797441908499853,
 1.1412173907712506,
 1.1597331186799331,
 1.1581282012211667,
 1.1337138783909562,
 1.1490233526383538,
 1.131429110618971,
 1.128879553500952,
 1.113899121145533,
 1.0996269047588507,
 1.1099622829314606,
 1.1829775011837143,
 1.1147937429283397,
 1.111827877376218,
 1.111827877376218,
 1.1567624058925778,
 1.1798151369736767,
 1.1856587261101108,
 1.1738752948827234,
 1.145541392877251,
 1.1381925184914443,
 1.2325322181955296,
 1.2445356648445816,
 1.2546410058199584,
 1.2538106045339144,
 1.2648322250722956,
 1.2648322250722956,
 1.2481585424840114,
 1.22461075069128,
 1.22461075069128,
 1.2505954724621449,
 1.2505954724621449,
 1.2506022174716085,
 1.2130083832585543,
 1.2130083832585543,
 1.2130083832585543,
 1.2130083832585543,
 1.2130083832585543,
 1.1998497086357418,
 1.1998497086357418,
 1.211991819513256,
 1.211991819513256,
 1.2

In [17]:
from pandas import Series
s = Series(result, index)
s.sort_values(ascending=False).head()

7     1.264970
34    1.264832
33    1.264832
31    1.254641
32    1.253811
dtype: float64

In [20]:
# 이동평균 계산
def short_long_single_moving_avg_ror(df,Ns,Nl):
    df = df[ [ '종가' ] ].copy()
    df['ma_s'] = df['종가'].rolling(Ns).mean().shift(1)
    df['ma_l'] = df['종가'].rolling(Nl).mean().shift(1)
    
    cond = (df['ma_s'] > df['ma_l']) & (df['ma_l'].pct_change() > 0)
    df['status'] = np.where(cond, 1, 0)
    df.iloc[-1,-1] = 0
    
    buy_condition = (df['status'] == 1) & (df['status'].shift(1) != 1)
    sell_condition = (df['status'] == 0) & (df['status'].shift(1) == 1)
    
    ROR = df.loc[sell_condition, '종가'].reset_index(drop=True) / df.loc[buy_condition, '종가'].reset_index(drop=True)
    ROR = ROR - 0.002
    return ROR.cumprod().iloc[-1]

In [21]:
short_long_single_moving_avg_ror(df,3,30)

1.1894497583801074

In [27]:
# 과체적화
result = []
for i in range(2,17):
    for j in range(30,61):
        result.append(short_long_single_moving_avg_ror(df,i,j))
    
index = pd.MultiIndex.from_product([range(2,17), range(30,61)])
s = Series(result, index)
print(s.idxmax())
print(s.max())

(14, 30)
1.237183001994187
